In [1]:
using Oceananigans
using Oceananigans.Units: minutes, hour, hours, day
using Random
using Printf
using CUDA

In [16]:
]st

Status `~/.julia/environments/v1.9/Project.toml`
⌅ [79e6a3ab] Adapt v3.7.2
⌅ [052768ef] CUDA v4.4.1
⌃ [13f3f980] CairoMakie v0.11.9
⌃ [a93c6f00] DataFrames v1.6.1
⌃ [39dd38d3] Dierckx v0.5.3
⌃ [9a22fb26] GibbsSeaWater v0.1.3
  [c27321d9] Glob v1.3.1
⌅ [7073ff75] IJulia v1.26.0
  [a98d9a8b] Interpolations v0.15.1
⌅ [033835bb] JLD2 v0.4.46
⌃ [da04e1cc] MPI v0.20.19
⌅ [30363a11] NetCDF v0.11.8
  [9e8cae18] Oceananigans v0.90.0 `https://github.com/CliMA/Oceananigans.jl.git#ss/distributed-fft`
⌃ [d0ccf422] Oceanostics v0.13.2
⌃ [91a5bcdd] Plots v1.39.0
⌃ [f27b6e38] Polynomials v4.0.6
⌃ [f2b01f46] Roots v2.1.2
⌃ [d496a93d] SeawaterPolynomials v0.3.4
⌃ [90137ffa] StaticArrays v1.9.3
  [10745b16] Statistics v1.9.0
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [2]:
grid = RectilinearGrid(CPU(), size=(128, 128, 34), extent=(3840, 3840,1020), halo=(3, 3, 3), topology=(Periodic, Periodic, Bounded))

128×128×34 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 3×3×3 halo
├── Periodic x ∈ [0.0, 3840.0)  regularly spaced with Δx=30.0
├── Periodic y ∈ [0.0, 3840.0)  regularly spaced with Δy=30.0
└── Bounded  z ∈ [-1020.0, 0.0] regularly spaced with Δz=30.0

In [3]:
buoyancy = SeawaterBuoyancy(equation_of_state=LinearEquationOfState(thermal_expansion = 1.19e-4,
                                                                    haline_contraction = 7.7e-4))

SeawaterBuoyancy{Float64}:
├── gravitational_acceleration: 9.80665
└── equation of state: LinearEquationOfState(thermal_expansion=0.000119, haline_contraction=0.00077)

In [4]:
T_flux(x, y, t, params) = params.initial_H_flux / (params.ρₒ * params.cᴾ) + 
                       (params.Hr * randn() / (params.ρₒ * params.cᴾ)) * exp(-t^4 / (24 * params.shut_off_time^4))  # K m s⁻¹

T_flux_parameters = (initial_H_flux = 300.0, ρₒ = 1027.62, cᴾ = 3991.0, Hr = 90,
                                    shut_off_time = 2hours)

T_flux_bc = FluxBoundaryCondition(T_flux, parameters = T_flux_parameters)

FluxBoundaryCondition: ContinuousBoundaryFunction T_flux at (Nothing, Nothing, Nothing)

In [5]:
dTdz = 3.2e-4 # ᵒC m⁻¹

T_gradient_bc = GradientBoundaryCondition(dTdz)

GradientBoundaryCondition: 0.00032

In [6]:
T_bcs = FieldBoundaryConditions(top = T_flux_bc,
                                bottom = T_gradient_bc)

Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.00032
├── top: FluxBoundaryCondition: ContinuousBoundaryFunction T_flux at (Nothing, Nothing, Nothing)
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)

In [7]:
coriolis = FPlane(f=1.4e-4) #1.4e-4

FPlane{Float64}(f=0.00014)

In [8]:
model = NonhydrostaticModel(; grid, buoyancy,
                            advection = UpwindBiased(order=5),
                            tracers = (:T, :S),
                            coriolis = coriolis,
                            closure = AnisotropicMinimumDissipation(),
                            boundary_conditions = (T=T_bcs,))

NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 128×128×34 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 3×3×3 halo
├── timestepper: QuasiAdamsBashforth2TimeStepper
├── tracers: (T, S)
├── closure: AnisotropicMinimumDissipation{Oceananigans.TurbulenceClosures.ExplicitTimeDiscretization, NamedTuple{(:T, :S), Tuple{Float64, Float64}}, Float64, Nothing}
├── buoyancy: SeawaterBuoyancy with g=9.80665 and LinearEquationOfState(thermal_expansion=0.000119, haline_contraction=0.00077) with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.00014)

In [9]:
# Random noise damped at top and bottom
Ξ(z) = randn() * z / model.grid.Lz * (1 + z / model.grid.Lz) # noise

# Temperature initial condition: a stable density gradient with random noise superposed.
Tᵢ(x, y, z) = z > -450 ? 5.5 : 5.495 + dTdz * (z + 450) # + dTdz * model.grid.Lz * 1e-6 * Ξ(z)

# Velocity initial condition: random noise scaled by the friction velocity.
uᵢ(x, y, z) = 1e-5 * Ξ(z)

# `set!` the `model` fields using functions or constants:
set!(model, u=uᵢ, w=uᵢ, T=Tᵢ, S=35)

In [10]:
simulation = Simulation(model, Δt=90.0, stop_time=5day)

Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 1.500 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 5 days
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [11]:
# Print a progress message
progress_message(sim) = @printf("Iteration: %04d, time: %s, Δt: %s, max(|w|) = %.1e ms⁻¹, wall time: %s\n",
                                iteration(sim), prettytime(sim), prettytime(sim.Δt),
                                maximum(abs, sim.model.velocities.w), prettytime(sim.run_wall_time))

simulation.callbacks[:progress] = Callback(progress_message, IterationInterval(50))

Callback of progress_message on IterationInterval(50)

In [12]:
fields = Dict("u" => model.velocities.u, "v" => model.velocities.v, "w" => model.velocities.w, "T" => model.tracers.T)

# %%
file_name = "DS96_solution_v1"

simulation.output_writers[:field_writer] =
    NetCDFOutputWriter(model, fields,
          filename = joinpath(@__DIR__, file_name * ".nc"),
          schedule = TimeInterval(6hour), overwrite_existing = true)

┌ Warning: Overwriting existing /data/gpfs/projects/punim1661/DS96_varification/DS96_solution_v1.nc.
└ @ Oceananigans.OutputWriters ~/.julia/packages/Oceananigans/Z5znR/src/OutputWriters/netcdf_output_writer.jl:359


NetCDFOutputWriter scheduled on TimeInterval(6 hours):
├── filepath: /data/gpfs/projects/punim1661/DS96_varification/DS96_solution_v1.nc
├── dimensions: zC(34), zF(35), xC(128), yF(128), xF(128), yC(128), time(0)
├── 4 outputs: (v, w, T, u)
└── array type: Array{Float64}

In [13]:
T_avg = Field(Average(model.tracers.T, dims=(1, 2)))
T_p = model.tracers.T - T_avg
T_p2 = Field(Average(T_p*T_p, dims=(1, 2)))

v_avg = Field(Average(model.velocities.v, dims=(1, 2)))
u_avg = Field(Average(model.velocities.u, dims=(1, 2)))
u_p = model.velocities.u - u_avg
u_p2 = Field(Average(u_p*u_p, dims=(1, 2)))

w_avg = Field(Average(model.velocities.w, dims=(1, 2)))
w_p = model.velocities.w - w_avg
w_p2 = Field(Average(w_p*w_p, dims=(1, 2)))

wT_p = Field(Average(w_p*T_p, dims=(1, 2)))

1×1×35 Field{Nothing, Nothing, Face} reduced over dims = (1, 2) on RectilinearGrid on CPU
├── data: OffsetArrays.OffsetArray{Float64, 3, Array{Float64, 3}}, size: (1, 1, 35)
├── grid: 128×128×34 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 3×3×3 halo
├── operand: mean! over dims (1, 2) of BinaryOperation at (Center, Center, Face)
└── status: time=0.0

In [14]:
simulation.output_writers[:avg_c] = NetCDFOutputWriter(model, (; u_avg=u_avg, v_avg=v_avg, w_avg=w_avg, T_avg=T_avg, u_p2=u_p2, w_p2=w_p2, T_p2=T_p2, wT_p=wT_p,),
                                                     filename = joinpath(@__DIR__, file_name * "_averaged.nc"),
                                                     schedule = TimeInterval(0.25hour), overwrite_existing = true)

NetCDFOutputWriter scheduled on TimeInterval(15 minutes):
├── filepath: /data/gpfs/projects/punim1661/DS96_varification/DS96_solution_v1_averaged.nc
├── dimensions: zC(34), zF(35), xC(128), yF(128), xF(128), yC(128), time(0)
├── 4 outputs: (u_p2, T_p2, w_p2, wT_p)
└── array type: Array{Float64}

In [15]:
run!(simulation)

[ Info: Initializing simulation...


Iteration: 0000, time: 0 seconds, Δt: 1.500 minutes, max(|w|) = 8.7e-06 ms⁻¹, wall time: 0 seconds


[ Info:     ... simulation initialization complete (30.974 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (3.777 seconds).


Iteration: 0050, time: 1.250 hours, Δt: 1.500 minutes, max(|w|) = 2.2e-03 ms⁻¹, wall time: 1.417 minutes
Iteration: 0100, time: 2.500 hours, Δt: 1.500 minutes, max(|w|) = 1.5e-02 ms⁻¹, wall time: 2.129 minutes
Iteration: 0150, time: 3.750 hours, Δt: 1.500 minutes, max(|w|) = 4.4e-02 ms⁻¹, wall time: 2.769 minutes
Iteration: 0200, time: 5 hours, Δt: 1.500 minutes, max(|w|) = 6.9e-02 ms⁻¹, wall time: 3.409 minutes
Iteration: 0250, time: 6.250 hours, Δt: 1.500 minutes, max(|w|) = 7.6e-02 ms⁻¹, wall time: 4.052 minutes
Iteration: 0300, time: 7.500 hours, Δt: 1.500 minutes, max(|w|) = 8.3e-02 ms⁻¹, wall time: 4.695 minutes
Iteration: 0350, time: 8.750 hours, Δt: 1.500 minutes, max(|w|) = 8.8e-02 ms⁻¹, wall time: 5.339 minutes
Iteration: 0400, time: 10 hours, Δt: 1.500 minutes, max(|w|) = 8.7e-02 ms⁻¹, wall time: 5.984 minutes
Iteration: 0450, time: 11.250 hours, Δt: 1.500 minutes, max(|w|) = 8.3e-02 ms⁻¹, wall time: 6.629 minutes
Iteration: 0500, time: 12.500 hours, Δt: 1.500 minutes, max(|

[ Info: Simulation is stopping after running for 1.047 hours.
[ Info: Simulation time 5 days equals or exceeds stop time 5 days.


Iteration: 4800, time: 5 days, Δt: 1.500 minutes, max(|w|) = 9.7e-02 ms⁻¹, wall time: 1.047 hours
